In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import numpy as np
import pandas as pd
import os

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor

# local import
from make_datasets import make_data
import models

2023-08-11 13:30:14.660600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = '/Users/jyontika/Desktop/jyontika-MA-data/data'
data_path= os.path.join(data_dir, './clean_annual_tract/')
data_gdf = gpd.read_file(data_path)

Process dataframe into a data frame with a Multiindex on location and time

In [3]:

# Name the important columns
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

# These are the columns we could possibly want in the X dataframe
x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']

# These are the columns we could want in the Y dataframe
y_idx_cols = [geography_col, timestep_col, outcome_col]

# These are the features we want
features_only = ['deaths']
add_spacetime = True
add_svi = True
if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


validation_year = 2018
first_test_year = 2019
last_test_year = 2020
first_test_timestep = 19
last_test_timestep = 20
lookback_years= 5
first_train_eval_year = validation_year - lookback_years
last_train_eval_year = validation_year -1

In [4]:
# Create the multiindex
multiindexed_gdf = data_gdf.set_index([geography_col, timestep_col])

# re-add the timestep column as a feature because it's useful
multiindexed_gdf[timestep_col] = multiindexed_gdf.index.get_level_values(timestep_col)

# Track number of locations
num_geoids = len(data_gdf[geography_col].unique())

In [5]:
x_BSF, y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          features_only, num_geoids)
x_test_BSF, y_test_BS =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          features_only, num_geoids)

# For the weighted historical average model, we only use deaths as features
x_BSF_death_only, y_BS_death_only = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          ['deaths'], num_geoids)
x_test_BSF_death_only, y_test_BS_death_only =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          ['deaths'], num_geoids)

In [19]:
x_test_BSF[0]

<tf.Tensor: shape=(1620, 45), dtype=float32, numpy=
array([[  1.      ,  42.05983 , -70.20041 , ...,   0.4262  ,   0.8232  ,
          0.6548  ],
       [  1.      ,  41.922634, -70.015366, ...,   0.0724  ,   0.5127  ,
          0.5216  ],
       [  0.      ,  42.013557, -70.06415 , ...,   0.1899  ,   0.207   ,
          0.2636  ],
       ...,
       [  0.      ,  42.239685, -71.701744, ...,   0.6209  ,   0.3016  ,
          0.2272  ],
       [  0.      ,  42.489754, -71.57967 , ...,   0.3928  ,   0.157   ,
          0.1229  ],
       [  0.      ,  42.53134 , -71.59275 , ...,   0.3928  ,   0.157   ,
          0.1229  ]], dtype=float32)>

In [6]:
x_BSF.shape

TensorShape([5, 1620, 45])

In [7]:
y_BS.shape

TensorShape([5, 1620])

In [8]:
#for rmse and mae
# # Assuming multiindexed_gdf contains the actual target predictions with 'year' and 'deaths' columns
actual_values_2019 = multiindexed_gdf[multiindexed_gdf['year'] == 2019]['deaths'].values
actual_values_2020 = multiindexed_gdf[multiindexed_gdf['year'] == 2020]['deaths'].values

### All Zeroes Model
#### lookback years =2

In [9]:
bpr_over_time_zeroes, predicted_over_time_zeroes = models.all_zeroes_model(multiindexed_gdf,
                                        first_test_timestep, last_test_timestep,
                                        num_geoids, bpr_uncertainty_samples=15)

In [10]:
print(f"2019 Average: {np.mean(bpr_over_time_zeroes[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_zeroes[0]) + \
                          np.array(bpr_over_time_zeroes[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.2588822943583979
Zeroes model (Mean, 95% CI): 25.4,
      (25.2-
       25.8)


In [11]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_over_time_zeroes[0]
predicted_samples_2020 = predicted_over_time_zeroes[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"Zeroes model RMSE for 2019: {rmse_2019:.2f}")
print(f"Zeroes model RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"Zeroes model MAE for 2019: {mae_2019:.2f}")
print(f"Zeroes model MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

Zeroes model RMSE for 2019: 1.75
Zeroes model RMSE for 2020: 1.86
Joint RMSE for 2019 and 2020: 1.81
 
Zeroes model MAE for 2019: 1.15
Zeroes model MAE for 2020: 1.21
Joint MAE for 2019 and 2020: 1.18


### Last Year
#### lookback = 1 year

In [12]:
bpr_over_time_last_time, predicted_over_time_last_time = models.last_time_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1,bpr_uncertainty_samples=50,)

In [13]:
print(f"2019 Average: {np.mean(bpr_over_time_last_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_last_time[0]) + \
                          np.array(bpr_over_time_last_time[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.5583207453088354
Zeroes model (Mean, 95% CI): 53.9,
      (52.3-
       55.4)


In [14]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_over_time_last_time[0]
predicted_samples_2020 = predicted_over_time_last_time[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"Last Year model RMSE for 2019: {rmse_2019:.2f}")
print(f"Last Year model RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"Last Year model MAE for 2019: {mae_2019:.2f}")
print(f"Last Year model MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

Last Year model RMSE for 2019: 1.50
Last Year model RMSE for 2020: 1.53
Joint RMSE for 2019 and 2020: 1.51
 
Last Year model MAE for 2019: 1.03
Last Year model MAE for 2020: 1.06
Joint MAE for 2019 and 2020: 1.04


### Historical Average 
#### lookback = 1 years

In [15]:
bpr_over_time_avg_time, predicted_over_time_avg_time = models.historical_average_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1, 7, bpr_uncertainty_samples=10,)

In [16]:
print(f"2019 Average: {np.mean(bpr_over_time_avg_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_avg_time[0]) + \
                          np.array(bpr_over_time_avg_time[1]))/2
                        
print(f"""Hist. Avg  model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6310450685551532
Hist. Avg  model (Mean, 95% CI): 61.0,
      (59.9-
       62.6)


In [17]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_over_time_avg_time[0]
predicted_samples_2020 = predicted_over_time_avg_time[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"Hist. Avg model RMSE for 2019: {rmse_2019:.2f}")
print(f"Hist. Avg model RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"Hist. Avg model MAE for 2019: {mae_2019:.2f}")
print(f"Hist. Avg model MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

Hist. Avg model RMSE for 2019: 1.17
Hist. Avg model RMSE for 2020: 1.24
Joint RMSE for 2019 and 2020: 1.21
 
Hist. Avg model MAE for 2019: 0.85
Hist. Avg model MAE for 2020: 0.89
Joint MAE for 2019 and 2020: 0.87


### Weighted Historical Average
#### lookback = 7 years

Make Scikit models

In [20]:
# Identical models, features are only difference
linear_poisson_weighted_avg = sklearn.linear_model.PoissonRegressor()
linear_poisson = sklearn.linear_model.PoissonRegressor()

# Params selected via grid search on validation. Need to re-do grid search for chicago
hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=3, max_leaf_nodes=2,
                                               l2_regularization=1, min_samples_leaf=100 )

In [21]:
bpr_over_time_weight_avg, predicted_over_time_weight_avg = models.scikit_model(multiindexed_gdf, x_BSF_death_only,
                                               y_BS_death_only, x_test_BSF_death_only,
                                               linear_poisson_weighted_avg,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [22]:
print(f"2019 Average: {np.mean(bpr_over_time_weight_avg[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_weight_avg[0]) + \
                          np.array(bpr_over_time_weight_avg[1]))/2
                        
print(f"""Weighted Hist. Avg  model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6264612161008314
Weighted Hist. Avg  model (Mean, 95% CI): 61.1,
      (59.2-
       63.4)


In [23]:
predicted_over_time_weight_avg

[array([0.70166866, 0.89404364, 0.70583855, ..., 0.65404996, 0.65404996,
        0.7522415 ]),
 array([0.7522415 , 0.87608488, 0.69889845, ..., 0.7522415 , 0.65404996,
        0.73828533])]

In [24]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_over_time_weight_avg[0]
predicted_samples_2020 = predicted_over_time_weight_avg[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"Weighted Hist. Avg model RMSE for 2019: {rmse_2019:.2f}")
print(f"Weighted Hist. Avg model RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"Weighted Hist. Avg model MAE for 2019: {mae_2019:.2f}")
print(f"Weighted Hist. Avg model MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

Weighted Hist. Avg model RMSE for 2019: 1.19
Weighted Hist. Avg model RMSE for 2020: 1.26
Joint RMSE for 2019 and 2020: 1.23
 
Weighted Hist. Avg model MAE for 2019: 0.90
Weighted Hist. Avg model MAE for 2020: 0.95
Joint MAE for 2019 and 2020: 0.93


### Linear (Poisson GLM baseline)

In [25]:
bpr_over_time_linear, predicted_over_time_linear = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               linear_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=50)

/Users/jyontika/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


In [26]:
print(f"2019 Average: {np.mean(bpr_over_time_linear[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_linear[0]) + \
                          np.array(bpr_over_time_linear[1]))/2
                        
print(f"""Linear (Poisson GLM) model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6320362020652359
Linear (Poisson GLM) model (Mean, 95% CI): 61.5,
      (59.8-
       63.6)


In [27]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_over_time_linear[0]
predicted_samples_2020 = predicted_over_time_linear[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"Linear (Poisson GLM) RMSE for 2019: {rmse_2019:.2f}")
print(f"Linear (Poisson GLM)  RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"Linear (Poisson GLM) MAE for 2019: {mae_2019:.2f}")
print(f"Linear (Poisson GLM) MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

Linear (Poisson GLM) RMSE for 2019: 1.26
Linear (Poisson GLM)  RMSE for 2020: 1.40
Joint RMSE for 2019 and 2020: 1.33
 
Linear (Poisson GLM) MAE for 2019: 0.99
Linear (Poisson GLM) MAE for 2020: 1.10
Joint MAE for 2019 and 2020: 1.05


### Gradient Boosted Trees (Poisson)

In [25]:
bpr_over_time_tree, predicted_over_time_tree = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               hist_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [26]:
print(f"2019 Average: {np.mean(bpr_over_time_tree[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_tree[0]) + \
                          np.array(bpr_over_time_tree[1]))/2
                        
print(f"""Gradient Boosted Trees (Poisson)  (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.5583002425125425
Gradient Boosted Trees (Poisson)  (Mean, 95% CI): 55.2,
      (53.4-
       57.3)


In [27]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_over_time_tree[0]
predicted_samples_2020 = predicted_over_time_tree[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"Gradient Boosted Trees (Poisson)  RMSE for 2019: {rmse_2019:.2f}")
print(f"Gradient Boosted Trees (Poisson) RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"Gradient Boosted Trees (Poisson) MAE for 2019: {mae_2019:.2f}")
print(f"Gradient Boosted Trees (Poisson)  MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

Gradient Boosted Trees (Poisson)  RMSE for 2019: 1.18
Gradient Boosted Trees (Poisson) RMSE for 2020: 1.25
Joint RMSE for 2019 and 2020: 1.22
 
Gradient Boosted Trees (Poisson) MAE for 2019: 0.89
Gradient Boosted Trees (Poisson)  MAE for 2020: 0.92
Joint MAE for 2019 and 2020: 0.91


### CASTNet

In [48]:
# Call the castnet_model function to calculate BPR for CASTNet predictions
bpr_results_castnet, predicted_results_castnet = models.castnet_model(multiindexed_gdf, first_test_timestep, last_test_timestep, 
                                            num_geoids, bpr_uncertainty_samples=50)


In [49]:
print(f"2019 Average: {np.mean(bpr_results_castnet[0])}")

bpr_samples_both_years = (np.array(bpr_results_castnet[0]) + \
                          np.array(bpr_results_castnet[1]))/2
                        
print(f"""CASTNet model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")

2019 Average: 0.2796877101780102
CASTNet model (Mean, 95% CI): 29.0,
      (27.8-
       30.4)


In [54]:
# Calculate the model predictions for the years 2019 and 2020
predicted_samples_2019 = predicted_results_castnet[0]
predicted_samples_2020 = predicted_results_castnet[1]

# Calculate RMSE for the Zeroes model for the year 2019 and 2020
rmse_2019 = np.sqrt(np.mean((predicted_samples_2019 - actual_values_2019)**2))
rmse_2020 = np.sqrt(np.mean((predicted_samples_2020 - actual_values_2020)**2))

joint_rmse = np.mean([rmse_2019, rmse_2020])

print(f"CASTNet RMSE for 2019: {rmse_2019:.2f}")
print(f"CASTNet RMSE for 2020: {rmse_2020:.2f}")
print(f"Joint RMSE for 2019 and 2020: {joint_rmse:.2f}")
print(" ")

mae_2019_samples = np.mean(np.abs(predicted_samples_2019 - actual_values_2019))
mae_2020_samples = np.mean(np.abs(predicted_samples_2020 - actual_values_2020))

# Take the average MAE for each year
mae_2019 = np.mean(mae_2019_samples)
mae_2020 = np.mean(mae_2020_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2019, mae_2020])

print(f"CASTNet MAE for 2019: {mae_2019:.2f}")
print(f"CASTNet MAE for 2020: {mae_2020:.2f}")
print(f"Joint MAE for 2019 and 2020: {joint_mae:.2f}")

CASTNet RMSE for 2019: 2.07
CASTNet RMSE for 2020: 2.18
Joint RMSE for 2019 and 2020: 2.13
 
CASTNet MAE for 2019: 1.50
CASTNet MAE for 2020: 1.71
Joint MAE for 2019 and 2020: 1.60
